## 程式主要功能 
* one shot analysis

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

import tkinter
from tkinter import*
# Create Tk root
root = Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)
from tkinter import filedialog
%gui tk

import FFT
import compensation
import module_4D
# import modeling_white_card
import model
import white_card_std

# from modeling_white_card import modeling_white
# from modeling import modeling
import interpolation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

C:\Users\bioinspira\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
C:\Users\bioinspira\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fi

In [2]:
def data_loading():
    path = filedialog.askdirectory()
    print(path)
    return path

In [3]:
def plot_overlay(data_list, cal_channel='583nm #1', QC_channel='583nm #1', save_figure=True, offset=False):
    save_figure = save_figure
    offset = offset
    cal_channel = cal_channel
    QC_channel = QC_channel

    plot_data = []
    data_list = data_list

    if offset == True:
        trace_ppm = go.Scatter(y=df_30C[cal_channel]-df_30C.loc[1,cal_channel], name=cal_channel+' calibration')
        plot_data.append(trace_ppm)

        for i,j in enumerate(data_list):
            trace_ppm = go.Scatter(y=j[QC_channel]-j.loc[1,QC_channel], name=QC_channel+' QC{}'.format(i+10))
            plot_data.append(trace_ppm) 
    else:
        trace_ppm = go.Scatter(y=df_30C[cal_channel], name=cal_channel+' calibration')
        plot_data.append(trace_ppm)

        for i,j in enumerate(data_list):
            trace_ppm = go.Scatter(y=j[QC_channel], name=QC_channel+' QC{}'.format(i+10))
            plot_data.append(trace_ppm)


    title = '{} raw data overlay'.format(QC_channel)    
    layout = go.Layout(title=title)
    fig = go.Figure(data=plot_data, layout=layout)
    fig.show()

    if save_figure == True:
        # save figure
        savePath = os.path.join(os.getcwd(), folder_name)
        filename = title
        fig.write_image(savePath + '/' + filename + '.jpeg', width=1280, height=720)

In [4]:
def response_separation(data_list, QC_channel='583nm #1', save_figure=True, humidity_step=1):
    """ data_list: list, dataframe
        QC_channel: str, channel name
        save_figure: bool
        humidity_step: int"""
    
    save_figure = save_figure
    QC_channel = QC_channel
    selected_range = { 
                       1:(4, 9, 14), 
                       2:(19, 24, 29),
                       3:(34, 39, 44),
                                                }

    diff_list = []
    for i,data in enumerate([df_30C]+test_data_list):
        diff = []
        a,b,c = selected_range.get(humidity_step)
        diff_b = data.loc[b-3:b, QC_channel].mean()-data.loc[a-3:a, QC_channel].mean()
        diff.append(abs(round(diff_b, 0)))
        diff_c = data.loc[c-3:c, QC_channel].mean()-data.loc[a-3:a, QC_channel].mean()
        diff.append(abs(round(diff_c, 0)))
        diff_list.append(diff)
        
    index = ['calibration']+['QC{}'.format(i+1) for i in range(0, len(test_data_list))]
    by_diff = pd.DataFrame(diff_list, columns=['60ppm-10ppm','40ppm-10ppm'], index=index)
    
    temperature = data.loc[a, 'Temperature']
    humidity = data.loc[a, 'Humidity']
    filename = '{} response separation (baseline 10ppm) ({}C {}%)'.format(QC_channel, int(temperature), int(humidity))
    
    
    plot_data = []
    trace = go.Bar(x=by_diff.index, y=by_diff.loc[:,'60ppm-10ppm'], name='60ppm-10ppm', text=by_diff.loc[:,'60ppm-10ppm'], textposition='auto')
    plot_data.append(trace)
    trace = go.Bar(x=by_diff.index, y=by_diff.loc[:,'40ppm-10ppm'], name='40ppm-10ppm', text=by_diff.loc[:,'40ppm-10ppm'], textposition='auto')
    plot_data.append(trace)

    layout = go.Layout(title=filename)
#     layout = go.Layout(title='{} response separation (baseline 4ppm)'.format(QC_channel))
    fig = go.Figure(data=plot_data, layout=layout)
    fig.update_layout()
    fig.show()

    if save_figure == True:
        # save figure
        savePath = os.path.join(os.getcwd(), folder_name)

        if not os.path.exists(savePath):
                os.makedirs(savePath)

        filename = '{} response separation (baseline 10ppm)'.format(QC_channel)
        fig.write_image(savePath + '/' + filename + '.jpeg', width=1280, height=720)

In [5]:
path = data_loading()

# 收尋path裡所有的資料夾、檔案的路徑
dirPath_list = []
dirNames_list = []
for dirPath, dirNames, _ in os.walk(path):
    dirPath_list.append(dirPath)
    dirNames_list.append(dirNames)

# 合成各個sensor路徑
path_list = []
for sensor in dirNames_list[0]:
    if '#20' in sensor:
        sensor_path = os.path.join(dirPath_list[0],sensor).replace('\\', '/')
        path_list.append(sensor_path)
path_list

D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)


['D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095021',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095051',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095121',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095135',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095155',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095215',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095426',
 'D:/Data_store/AMS_7341/111822 FPCB poultry (batch 09-23-22-2)(new chip process)(probe KQ)/#20221107095440']

In [6]:
# path_list.pop(-3)

In [7]:
# sensor_number = 1
# a = 1
# path_list = []
# while a <= sensor_number:
#     path = data_loading()
#     path_list.append(path)
#     a += 1

In [8]:
# Sensor model setting 
sensor_humidity_feature = True
interpolation_model = 'linear'
model_name = Ridge(alpha=10)                    #regression model
shift = 0                             #raw data signal shifting
multiple = 1                                    #raw data ppm multiple

train_channel = 'Filtered_Z_score_583nm #1'                   #training channel
test_channel = 'Filtered_Z_score_583nm #1'                    #testing channel

degree = 2                                      #polynimial degree
output_modify = 1                               #output multiple


# folder_name = 'QC1(10,20,30,40C)({})(degree {})({})'.format(interpolation_model, degree, train_channel)

# folder_name = 'Train_30C'
# folder_name = 'Train_30C (clear channel compensation)'
folder_name = 'QC1'
# folder_name = 'QC1 (clear channel compensation)'
# folder_name = 'QC2'
# folder_name = 'QC2 (clear channel compensation)'
# folder_name = 'QC3'
# folder_name = 'QC3 (clear channel compensation)'
# folder_name = 'QC4'

In [9]:
def moving_average(data, windows=1, windows_offset=1):
    '''data: list(int, float)
       windows: int, 
       windows_offset: int'''
    
    moving_average_result = []
    i = 0
    while i < len(data):
        average = round(data[i:i+windows].mean(),1)
        moving_average_result.append(average)
        i += windows_offset
    return moving_average_result

In [10]:
#sensor serial number
for path in path_list:      
    sensor_number = path.split('/')[-1]
    print('sensor_number:', sensor_number)
    
# load training data
    A = 0
    B = 300000

    df_10C = (pd.read_csv(path + '/select_data_calibration_10C.csv').reset_index(drop=True))
#     df_20C = (pd.read_csv(path + '/select_data_calibration_20C.csv').reset_index(drop=True))
    df_30C = (pd.read_csv(path + '/select_data_calibration_30C.csv').reset_index(drop=True))
    df_40C = (pd.read_csv(path + '/select_data_calibration_40C.csv').reset_index(drop=True))
        
    train_data_list = [
                        df_10C, 
#                         df_20C, 
                        df_30C,
                        df_40C,
                        ]

# Load testing data

    df_QC1 = (pd.read_csv(path + '/select_data_blind_test_30C_1.csv')[A:B].reset_index(drop=True))
#     df_QC2 = (pd.read_csv(path + '/select_entire_data_QC2 (0ppm diffusion ball)(102822).csv')[A:B].reset_index(drop=True))
#     df_QC3 = (pd.read_csv(path + '/select_data_blind_test_30C_3.csv')[A:B].reset_index(drop=True))
#     df_QC4 = (pd.read_csv(path + '/select_data_blind_test_30C_4.csv')[A:B].reset_index(drop=True))
    test_data_list = [ 
#                       df_30C,
                      df_QC1,
#                       df_QC2,
#                       df_QC3,
#                         df_QC4,
                     ]

# white card paremeter setting 
    
    model_name = Ridge(alpha=10)                    #regression model
    channel = '440nm #1'                       #regression channel
    degree_white_card = 2                                   #polynimial degree
    humidity_step = ['RH40%', 'RH50%', 'RH60%', 'RH70%', 'RH80']
    white_card_humidity_feature = False

    df_all = pd.concat(train_data_list, ignore_index=True)
    df_test = pd.concat([df_10C]).reset_index(drop=True)

    y_train = df_all[channel]
    x_train = df_all[['Temperature', 'Humidity']]
    y_test = df_test[channel]
    x_test = df_test[['Temperature', 'Humidity']]

    os.chdir(path) 
    white_card = model.model(folder_name=folder_name,
                             sensor_number=sensor_number,
                             channel=channel,
                             x_train=x_train, 
                             y_train=y_train, 
                             x_test=x_test, 
                             y_test=y_test, 
                             model_name=model_name, 
                             degree=degree_white_card,                        
                             ppm=-23,
                             humidity_feature=white_card_humidity_feature)

    white_intercept, white_coef = white_card.regression()
    white_card.save_white_card_side_coef()
#     white_card.plot()
#     white_intercept, white_coef = white_card.coef()


    # concate training data        
    df_all = pd.concat(train_data_list).reset_index(drop=True)
    df_test = pd.concat(test_data_list).reset_index(drop=True)   

    # 取0ppm資料
    dk_all_0ppm = df_all[df_all['ppm']<1].copy()

    # 取4ppm資料
#     dk_all_4ppm = df_all[(df_all['ppm']>3)&(df_all['ppm']<5)].copy()

    # 取10ppm資料
    dk_all_10ppm = df_all[(df_all['ppm']>9)&(df_all['ppm']<11)].copy()

    # 取40ppm資料
    dk_all_40ppm = df_all[(df_all['ppm']>39)&(df_all['ppm']<41)].copy()

    # 取60ppm資料
    dk_all_60ppm = df_all[(df_all['ppm']>59)&(df_all['ppm']<61)].copy()
    
    # 取80ppm資料
    dk_all_80ppm = df_all[(df_all['ppm']>79)&(df_all['ppm']<81)].copy()


    dk_data_dict = {
                    10:(dk_all_10ppm),
                    40:(dk_all_40ppm),
                    60:(dk_all_60ppm),
               }


    #training model for 10ppm, 40ppm, 60ppm surface
    surf_coef_list = []                           # store ppm surface coefficient
    for ppm, ppm_data in dk_data_dict.items():

        dk_data = ppm_data
        y_train = dk_data[train_channel] + shift 
        x_train = dk_data[['Temperature', 'Humidity']]
        y_test = df_test[test_channel] 
        x_test = df_test[['Temperature', 'Humidity']]

#             channels = train_channel

        #開始計算濃度平面 model by Ridge regression
        os.chdir(path)              
        loss_list = []
        loss_each_step_list = []
        sensor = model.model(folder_name=folder_name,
                             sensor_number=sensor_number,
                             channel=train_channel,
                             x_train=x_train, 
                             y_train=y_train, 
                             x_test=x_test, 
                             y_test=y_test, 
                             model_name=model_name, 
                             degree=degree,
                             output_modify=output_modify,
                             shift=shift,            
                             multiple=multiple,
                             ppm=ppm,
                             humidity_feature=sensor_humidity_feature                             
                             )
        sensor.regression()
        intercept_and_coefficient = sensor.save_sensor_side_coef()
#         print(intercept_and_coefficient)
#         sensor.loss()
#         sensor.plot()
#         sensor.save_plot()
#         sensor.coef()

        surf_coef_list.append(intercept_and_coefficient) 

    test_data = df_test[[test_channel, 'Temperature', 'Humidity', 'ppm']]
    
    ppm_list = []
    for index, row in test_data.iterrows():
        signal, temperature, humidity, _ = row
#         signal_0ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[0], x0=temperature, x1=humidity, degree=degree)
        signal_10ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[0], x0=temperature, x1=humidity, degree=degree)
        signal_40ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[1], x0=temperature, x1=humidity, degree=degree)
        signal_60ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[2], x0=temperature, x1=humidity, degree=degree)
#         signal_60ppm = interpolation.calculate_ppm_surface_signal(surf_coef_list[4], x0=temperature, x1=humidity, degree=degree)


#         print(signal_10ppm, signal_40ppm, signal_60ppm)

        signal_ppm_dict = {
#                              0:signal_0ppm,
                             10:signal_10ppm,
                            40:signal_40ppm,
                            60:signal_60ppm,
#                             60:signal_60ppm,
                           }

        if interpolation_model == 'linear':
            ppm = interpolation.linear_interpolation(signal_ppm_dict, signal, 10, 40, 60)

        elif interpolation_model == 'log':
            ppm = interpolation.log_interpolation(signal_ppm_dict, signal, 10, 40, 60)

        elif interpolation_model == 'hybrid':
            ppm = interpolation.hybrid_interpolation(signal_ppm_dict, signal, 10, 40, 60)

        ppm_list.append(ppm)

    # RMSE
    rmse_test = np.sqrt(mean_squared_error(ppm_list, test_data['ppm'])).round(5)

    # Interpolation prediction result    
    predict_test_data = test_data.copy()
    predict_test_data['pred_ppm'] = ppm_list
    temp = int(predict_test_data['Temperature'].mean())
    hum_min = int(predict_test_data['Humidity'].min())
    hum_max = int(predict_test_data['Humidity'].max())
    title = '({}) ({}%~{}%) ({}) (RMSE={})'.format(sensor_number, hum_min, hum_max, test_channel, rmse_test)
#     title = '({}) ({}%~{}%) ({}) (RMSE={})'.format(sensor_number, hum_min, hum_max, test_channel, 0)

    # plot prediction result  
    fig_sub = make_subplots(rows=4, cols=1,
                            vertical_spacing=0.1,
                            row_heights=[40,40,40,120],
                            subplot_titles= ['Humidity', 'Temperature','Prediction', 'ppm error'],
                            )
    
    
    fig_sub.add_trace(go.Scatter(y=predict_test_data['Humidity'], name='Humidity'), row=1, col=1)
    fig_sub.add_trace(go.Scatter(y=predict_test_data['Temperature'], name='Temperature'), row=2, col=1)
    fig_sub.add_trace(go.Scatter(y=predict_test_data['ppm'], name='ppm'), row=3, col=1)
#     fig_sub.add_trace(go.Scatter(y=[0]*len(predict_test_data), name='ppm'), row=3, col=1)
    fig_sub.add_trace(go.Scatter(y=predict_test_data['pred_ppm'], name='pred_ppm'), row=3, col=1)

    
    ppm_error = predict_test_data['pred_ppm'] - predict_test_data['ppm']
    ppm_error_moving_average = moving_average(ppm_error, windows=5, windows_offset=5)
    
    fig_sub.add_trace(go.Bar(y=ppm_error_moving_average, name='ppm error',
                             text=ppm_error_moving_average, textposition='auto'), row=4, col=1)

#     fig_sub.add_vrect(x0=0, x1=46, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0)
#     fig_sub.add_vrect(x0=882, x1=937, fillcolor="LightSalmon", opacity=0.5, layer="below", line_width=0)

    save_path = os.path.join(os.getcwd(), folder_name)
    filename = '({}%~{}%)({})(shift {})(RMSE={})'.format(hum_min, hum_max, test_channel, shift ,rmse_test)
    fig_sub.update_layout(yaxis4 = dict(range=[min(ppm_error_moving_average)-6, max(ppm_error_moving_average)+6]))
    fig_sub.update_layout(title=filename ,width=900, height=800, legend_tracegroupgap=135)
    fig_sub.write_image(save_path + '/' + filename + '.jpeg')
    fig_sub.show()
    
#     # cap ppm value (replace below 0 value as 0)
#     np_ppm_list = np.array(ppm_list)
#     predict_test_data['pred_ppm_cap'] = np.where(np_ppm_list>0, np_ppm_list, 0)
    
#     # save prediction result by temperature step
#     predict_test_data['temp_step'] = predict_test_data['Temperature'].apply(lambda x: int(round(x/100, 1)*100) if abs(x) > 10 else int(round(x,0)))
#     temp_step = predict_test_data['temp_step'].unique()
#     print("Temperature step: ", temp_step)
    
#     for temp in temp_step:
#         predict_test_data_by_temp = predict_test_data[predict_test_data['temp_step']==temp]
#         if predict_test_data_by_temp.shape[0] > 5:
#             predict_test_data_by_temp.to_csv(save_path + '/predict_result_{}C.csv'.format(temp), index=False)
    

# # Overview 4D data
# # 4D plot
#     training_data = {
# #                  '0C':df_0C,
#                  '10C':df_10C,
#                  '20C':df_20C,
#                  '30C':df_30C,
#                  '40C':df_40C,
#                      }

#     testing_data = {
#                  '30C_test_10':df_test_30C_10,
# #                  '10C_test_2':df_test_10C_2,
# #                  '10C_test_3':df_test_10C_3,

#                    }

#     channel_list=['Filtered_583nm_Z_score']
#     module_4D.plot_4D(training_data, testing_data, channel_list, sensor_number)

# Analysis data 
    
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_620nm_Z_score', QC_channel='Filtered_620nm_Z_score', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_583nm_Z_score', QC_channel='Filtered_583nm_Z_score', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_670nm_Z_score', QC_channel='Filtered_670nm_Z_score', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_510nm_Z_score', QC_channel='Filtered_510nm_Z_score', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_550nm_Z_score', QC_channel='Filtered_550nm_Z_score', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_470nm_Z_score', QC_channel='Filtered_470nm_Z_score', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_583nm_Z_score', QC_channel='Filtered_583nm_Z_score', save_figure=True, offset=False)
# 
#     plot_overlay(data_list=test_data_list, cal_channel='583nm #1', QC_channel='583nm #1', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_620nm #1', QC_channel='Filtered_620nm #1', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_470nm #1', QC_channel='Filtered_470nm #1', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_550nm #1', QC_channel='Filtered_550nm #1', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_670nm #1', QC_channel='Filtered_670nm #1', save_figure=True, offset=False)
#     plot_overlay(data_list=test_data_list, cal_channel='Filtered_510nm #1', QC_channel='Filtered_510nm #1', save_figure=True, offset=False)

#     response_separation(data_list=test_data_list, QC_channel='583nm #1', save_figure=True, humidity_step=2)
# 


sensor_number: #20221107095021


sensor_number: #20221107095051


sensor_number: #20221107095121


sensor_number: #20221107095135


sensor_number: #20221107095155


sensor_number: #20221107095215


sensor_number: #20221107095426


sensor_number: #20221107095440
